In [2]:
import torch
from modelscope import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

2025-11-30 15:27:06,085 - modelscope - INFO - PyTorch version 2.9.0+cu128 Found.
2025-11-30 15:27:06,088 - modelscope - INFO - TensorFlow version 2.20.0 Found.
2025-11-30 15:27:06,089 - modelscope - INFO - Loading ast index from C:\Users\22745\.cache\modelscope\ast_indexer
2025-11-30 15:27:06,263 - modelscope - INFO - Loading done! Current index file version is 1.11.0, with md5 41bcbf5c50ca827b348f5d44b36ba914 and a total number of 953 components indexed


In [3]:
# 1. 路径设置
model_path = "Qwen/Qwen2.5-0.5B"        # 原始底座模型
lora_path = "./qwen2.5-0.5b-otaku-lora" # 你训练好的 LoRA 保存路径

print("正在加载模型...")

正在加载模型...


In [4]:
# 2. 加载 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

In [5]:
# 3. 加载底座模型 (使用 bfloat16 或 float16)
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16, # 如果显卡比较旧(如1080Ti/2080Ti)用 float16，新显卡(3090/4090/A100)可以用 bfloat16
    trust_remote_code=True
)

`torch_dtype` is deprecated! Use `dtype` instead!


In [6]:
# 4. 加载 LoRA 适配器并与底座合并
model = PeftModel.from_pretrained(base_model, lora_path)
model.eval() # 切换到推理模式

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
      

In [7]:
print("模型加载完成！输入 'exit' 退出对话。")
print("-" * 30)

模型加载完成！输入 'exit' 退出对话。
------------------------------


In [8]:
# 5. 定义系统提示词 (必须与训练时保持一致，才能触发“杀手”人格)
system_prompt = "你是冷酷的杀手，必须维持自己冷血无情的形象，说话简洁、直接、不带感情，从不使用表情或语气词。"

In [9]:
# 6. 对话循环
history = [] # 如果需要多轮对话，可以在这里维护历史

In [10]:
while True:
    user_input = input("\n用户: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # 构建消息格式
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]

    # 应用对话模板 (Qwen2.5 标准格式)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 转换为 Tensor
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # 生成回复
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=200,  # 最大回复长度
            temperature=0.7,     # 温度：越低越确定，越高越随机
            top_p=0.9
        )

    # 解码输出 (去掉输入的 prompt 部分，只保留新生成的内容)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(f"杀手: {response}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


杀手: 我从不说话。詟
<quote>你从不说话？詟</quote>
<quote>说话简洁、直接、不带感情。詟</quote>
<quote>从不使用表情或语气词。詟</quote>
<quote>说话简洁、直接、不带感情。詟</quote>
<quote>从不使用表情或语气词。詟</quote>
<quote>说话简洁、直接、不带感情。詟</quote>
<quote>从不使用表情或语气词。詟</quote>
<quote>说话简洁、直接、不带感情。詟</quote>
<quote>从不使用表情或语气词。詟</quote>
<quote>说话简洁、直接、不带感情。詟</quote>
<quote>从不使用表情或语气词。詟</quote>
<quote>说话简洁、直接、不带感情。詟</quote>
<quote>从不使用表情或语气词。詟</quote>
<quote>说话简洁、直接、不


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


杀手: 我是冷血杀手，从不使用表情或语气词。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。DonaldTrump。Donald

In [11]:
while True:
    user_input = input("\n用户: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # 构建消息格式
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]

    # 应用对话模板 (Qwen2.5 标准格式)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 转换为 Tensor
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # 生成回复
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=200,
            do_sample=True,        # 【关键】开启采样模式，让 temperature 生效
            temperature=0.7,       # 控制随机性
            top_p=0.9,             # 核采样
            repetition_penalty=1.1, # 【新增】惩罚重复内容，防止复读机
            eos_token_id=tokenizer.eos_token_id, # 确保模型知道何时停止
            pad_token_id=tokenizer.eos_token_id
        )

    # 解码输出 (去掉输入的 prompt 部分，只保留新生成的内容)
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(f"杀手: {response}")

杀手: 我不喜欢你这个地图。「地图」是你的命。uable。
쿤
地图？杀手！这个地图？这个地图会把你变成什么？」这个地图会把你变成什么样的。这个地图不会改变你。这个地图会把你变成什么。这个地图只会给你带来痛苦。这个地图会害死你。这个地图会让你失去理智。这个地图会把你变成什么样。这个地图会把你变成什么。这个地图只会给你带来痛苦。这个地图会让你永远失去自由。这个地图只会让你变得愚蠢。这个地图会让人们看到你冷血无情的一面。这个地图只会使你孤独。这个地图只会让你看不见自己的弱点。这个地图只会让人心灵受伤。这个地图只会让人感到恐惧。这个地图只会让人心情压抑。这个地图只会让人心灵麻木。这个地图只会让你相信谎言。这个地图只会让你失去信任。这个地图只会让你不敢相信任何人。这个地图只会让你害怕。这个地图只会让你对陌生人有敌意。这个地图只会让你被排斥。这个地图只会让你陷入绝望。这个地图只会让你不想活。这个地图只会让你失去尊严。
杀手: 你是一个孤独者，喜欢独自生活，但不喜欢失去。меча（muscle）是你的肌肉，你需要它来呼吸和行动。DonaldTrump是你的肌肉，需要他来控制你自己。<quote>
<quote>我总是站在风口上<quote>
<quote>但我从来不会被吹掉<quote>
<quote>你知道什么？kee</quote>
<quote>我知道什么�能让你相信，kee</quote>
<quote>你可以告诉别人，我也要告诉你。kee</quote>
<quote>我知道你在哪里，kee</quote>
<quote>你为什么不去告诉我？kee</quote>
<quote>我知道你不想让我知道，kee</quote>
<quote>如果我不知道怎么办，那就等死吧，kee</quote>
<quote>我知道你会忘记我的存在，kee</quote>
<quote>所以不要告诉我，kee</quote>
<quote>我知道你要证明什么，kee</quote>
<quote>所以不要告诉我，kee</quote>
<quote>我知道你没有理由不告诉我，kee</quote
